In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#dataload
BRCA1_raw = pd.read_csv("BRCA1_HUMAN_Findlay_2018.csv")
#dataload, Marastyle
df = pd.read_csv("BRCA1_HUMAN_Findlay_2018.csv")

In [4]:
#code to split our dataframe into its domains

RING = BRCA1_raw[:575]
BRCT = BRCA1_raw[575:]

In [ ]:
# add column named: 'position'
import re 

def get_number_from_text(text):
    return re.findall(r'\d{1,4}',text)[0]

df['position']=df['mutant'].apply(get_number_from_text)
print(df)